In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('olist_orders_dataset.csv')
print(df.head())


                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

In [ ]:
df = df.dropna(subset=['order_id', 'customer_id', 'order_status'])

In [ ]:
# 4. Converter colunas de data
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

In [ ]:
# 5. Criar a pasta 'dados_tratados' se não existir
os.makedirs('dados_tratados', exist_ok=True)

In [ ]:
# 6. Salvar arquivo tratado
df.to_csv('dados_tratados/olist_orders_tratado.csv', index=False)
print("✅ Arquivo tratado salvo com sucesso!")

✅ Arquivo tratado salvo com sucesso!


# Tratamento olist_customers_dataset



In [ ]:
df_clientes = pd.read_csv("olist_customers_dataset.csv")

In [ ]:
dim_cliente = df_clientes[[
    "customer_id",
    "customer_unique_id",
    "customer_city",
    "customer_state"
]].drop_duplicates()

In [ ]:
os.makedirs("dimensoes", exist_ok=True)
dim_cliente.to_csv("dimensoes/dim_cliente.csv", index=False)

# Tratamento olist_products_dataset

In [ ]:
df_prod = pd.read_csv("olist_products_dataset.csv")

In [ ]:
df_cat = pd.read_csv("product_category_name_translation.csv")

In [ ]:
dim_produto = df_prod.merge(df_cat, how="left", on="product_category_name")
dim_produto = dim_produto[[
    "product_id",
    "product_category_name",
    "product_category_name_english",
    "product_name_lenght",
    "product_description_lenght",
    "product_weight_g"
]].drop_duplicates()

In [ ]:
os.makedirs("dimensoes", exist_ok=True)
dim_produto.to_csv("dimensoes/dim_produto.csv", index=False)

# Tratamento olist_sellers_dataset

In [ ]:
df_sellers = pd.read_csv("olist_sellers_dataset.csv")

In [ ]:
dim_vendedor = df_sellers[[
    "seller_id",
    "seller_city",
    "seller_state"
]].drop_duplicates()

In [ ]:
os.makedirs("dimensoes", exist_ok=True)

In [ ]:
dim_vendedor.to_csv("dimensoes/dim_vendedor.csv", index=False)


# Tratamento olist_orders_dataset

In [ ]:
df_orders = pd.read_csv("olist_orders_dataset.csv")


In [ ]:
dim_tempo = df_orders[["order_purchase_timestamp"]].dropna().drop_duplicates()
dim_tempo["data"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.date
dim_tempo["ano"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.year
dim_tempo["mes"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.month
dim_tempo["dia"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.day
dim_tempo["dia_semana"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.day_name()
dim_tempo["trimestre"] = pd.to_datetime(dim_tempo["order_purchase_timestamp"]).dt.quarter


In [ ]:
dim_tempo = dim_tempo.drop(columns=["order_purchase_timestamp"]).drop_duplicates()


In [ ]:
os.makedirs("dimensoes", exist_ok=True)
dim_tempo.to_csv("dimensoes/dim_tempo.csv", index=False)

# etl_fato_vendas

In [ ]:
df_items = pd.read_csv("olist_order_items_dataset.csv")

In [ ]:
df_orders = pd.read_csv("olist_orders_dataset.csv")

In [ ]:
df_payments = pd.read_csv("olist_order_payments_dataset.csv")

In [ ]:
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv")

In [ ]:
# Join principal: order_items + orders
fato_vendas = df_items.merge(df_orders, on="order_id", how="left")

In [ ]:
# Pagamentos: usar apenas um tipo por pedido
df_pagamentos_unicos = df_payments.drop_duplicates(subset=["order_id"])
fato_vendas = fato_vendas.merge(df_pagamentos_unicos, on="order_id", how="left")


In [ ]:
# Reviews: nota da avaliação
df_reviews_simples = df_reviews[["order_id", "review_score"]].drop_duplicates()
fato_vendas = fato_vendas.merge(df_reviews_simples, on="order_id", how="left")


In [ ]:
# Seleciona colunas da fato
fato_vendas = fato_vendas[[
    "order_id",
    "order_item_id",
    "product_id",
    "seller_id",
    "price",
    "freight_value",
    "payment_type",
    "payment_value",
    "review_score",
    "order_purchase_timestamp",
    "order_delivered_customer_date"
]]

In [ ]:
# Apenas salve diretamente, sem tentar criar a pasta
fato_vendas.to_csv("/content/fato_vendas.csv", index=False)


In [ ]:
# Salva o arquivo dentro de /content (que é o diretório atual do Colab)
fato_vendas.to_csv("fato_vendas.csv", index=False)

# Faz o download para sua máquina
from google.colab import files
files.download("fato_vendas.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📤 Importando 'dim_cliente.csv' para a tabela 'dim_cliente'...
📤 Importando 'dim_produto.csv' para a tabela 'dim_produto'...
📤 Importando 'dim_vendedor.csv' para a tabela 'dim_vendedor'...
📤 Importando 'dim_tempo.csv' para a tabela 'dim_tempo'...


634

✅ Tabela 'dim_tempo' populada com sucesso!

🏁 Todas as dimensões foram importadas para o banco.
